In [1]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from pandasticsearch import Select, DataFrame

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Notebook - Kabana lookalike

In [3]:
from IPython.display import Image
image="https://i0.wp.com/sisypheanhigh.com/foot/wp-content/uploads/2015/08/sisyphus-red.png"
Image(url=image)

In [4]:
hostname="http://noname-sms.us.cray.com:30200"
client =  Elasticsearch(hostname, http_compress=True)

In [5]:
cluster_health=client.cluster.health()
cluster_health

{'cluster_name': 'elasticsearch',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 6,
 'active_shards': 6,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 6,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 50.0}

In [6]:
client_info=client.info()
client

<Elasticsearch([{'host': 'noname-sms.us.cray.com', 'port': 30200}])>

## Read the data from elasticsearch

__Note:__ We use wildcard on the shasta-logs, so we are pulling from _ALL_ data sources

In [57]:
resp = client.search(
    index="shasta-logs-*",
    body={
        "size":0,
        "query": {
            "bool": {
              "must": [{
                  "match_all": {}
                },
                {
                  "range": {
                    "timereported": {
                      "gte": "2019-07-29",
                      "lte": "2019-07-29",
                      "format": "yyyy-MM-dd HH:mm:ss||yyyy-MM-dd||epoch_millis"
                }
              }
            }
          ],
          "must_not": []
        }
      },
        "_source":{
            "excludes":[]
        },
        "aggs": {
        "2": {
          "date_histogram": {
            "field": "timereported",
            "interval": "30s",
            "time_zone": "America/Chicago",
            "min_doc_count": 1
          }
        }
      }
    }
)

In [58]:
print("Number of responses:%d." % resp['hits']['total'])

Number of responses:5942356.


In [9]:
table=resp['aggregations']['2']['buckets']

In [10]:
pd_table=pd.DataFrame.from_dict(table)
pd_table.drop(columns="key", inplace=True)
#pd_table.drop(columns="key_as_string", inplace=True)
pd_table.head(5)

,key_as_string,doc_count
0,2019-07-29T17:33:30.000-05:00,1015
1,2019-07-29T17:34:00.000-05:00,1288
2,2019-07-29T17:34:30.000-05:00,1359
3,2019-07-29T17:35:00.000-05:00,1495
4,2019-07-29T17:35:30.000-05:00,1584


Generate some statistical report on the message counts.

In [11]:
pd_table['doc_count'].describe()

count      31.000000
mean     1437.129032
std       414.901092
min       470.000000
25%      1301.500000
50%      1359.000000
75%      1530.500000
max      3324.000000
Name: doc_count, dtype: float64

In [12]:
#pd_table.columns

In [13]:
#pd_table.shape

In [14]:
#pd_table.hist(column='doc_count', color='#86bf91',grid=False,bins=100)

In [15]:
# Bokeh libraries
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
from bokeh.models.tools import HoverTool
from bokeh.models import DatetimeTickFormatter
from math import pi

In [16]:
output_notebook()  # Render inline in a Jupyter Notebook

Loading BokehJS ...

In [17]:
#print(pd_table.head())
pd_table['key_as_string']=pd.to_datetime(pd_table['key_as_string'].astype(str), format='%Y-%m-%dT%H:%M:%S.%f%z')
#print(pd_table.head())

In [45]:
count=pd_table['doc_count'].count()
print("Number of entries: %d." % count)

Number of entries: 31.


In [48]:
if ( count > 100 ):
   sample = pd_table.sample(100)
else:
   sample = pd_table.sample(count)

X=sample['key_as_string'].tolist()
Y=sample['doc_count'].tolist()

In [55]:
p = figure(x_axis_type="datetime",plot_width=1200, plot_height=400)
source = ColumnDataSource(data={
    'datetime' : X,
    'count' : Y,
})
#p.circle(X, Y, size = 5, color = 'green')
p.circle(x='datetime',y='count', source=source, size = 5, color = 'green' )

p.title.text = 'Message Counts per 30 minutes'
p.background_fill_color="#f5f5f5"
p.grid.grid_line_color="white"
p.yaxis.axis_label = 'Count'
p.xaxis.axis_label =' timereported per 30 minutes'
p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )
p.xaxis.major_label_orientation = pi/4

hover = HoverTool(
    tooltips=[
        ('Count', '@count{%d}'),
        ('timereported per 30 seconds','@datetime{%Y-%m-%d %H:%M:%S.%3N}')
    ],
    formatters={
        'count':'printf',
        'datetime': 'datetime',
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline'
)

p.add_tools(hover)
show(p)

## Lets try a bar or vbar graph

In [54]:
# Set the x_range to the list of categories above
p2 = figure(x_axis_type="datetime",plot_width=1200, plot_height=400)
p2.vbar(x=Y, top=X, width=0.9)
# Set some properties to make the plot look better
p2.xgrid.grid_line_color = None
p2.y_range.start = 0
show(p2)